# Capstone Project: The battle of neighbourhoods

In [1]:
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

In [3]:
import pandas as pd
from pandas.io.html import read_html
Canada_Indians = 'https://en.wikipedia.org/wiki/Indo-Canadians#cite_note-statcan2011-12'

Indians = read_html(Toronto_Indians, attrs={'class':'wikitable'})

In [4]:
Canada_Indo = Indians[0]
Canada_Indo.head()

,City,Province,Indian,Percentage
0,Toronto,Ontario,643370,10.4%
1,Vancouver,British Columbia,243135,10.0%
2,Montréal,Quebec,48485,1.2%
3,Calgary,Alberta,90625,6.5%
4,Edmonton,Alberta,72245,5.5%


### As, in above Data Table we can find the Toronto city is the best city with maximum number of Indians.

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [11]:
from pandas.io.html import read_html
Toronto_Neighborhoods = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Neighborhood = read_html(Toronto_Neighborhoods, attrs={'class':'wikitable'})

In [12]:
canada_data = Neighborhood[0]

In [13]:
canada_data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [14]:
canada_data.shape

(180, 3)

In [15]:
canada_data.dropna(subset=["Borough","Neighborhood"], how='any',inplace= True)

In [16]:
canada_data.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [17]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [18]:
postal_data = pd.read_csv('http://cocl.us/Geospatial_data')
postal_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
toronto_data = pd.merge(canada_data,postal_data, on= 'Postal Code')
toronto_data.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [24]:
toronto_data.shape

(103, 5)

In [40]:
address = 'Toronto,TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [43]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [44]:
pd.value_counts(toronto_data['Borough'].values)

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
dtype: int64

### As, by above data analysis we get the best Borough in terms of maximum number of neighbourhoods. 

In [52]:
NorthYork_data = toronto_data[toronto_data['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [45]:
CLIENT_ID = 'ZBO2RSEX4F1LTHDGY32VNYOOL1XAI1HFIF5125S0MIVAPAUQ' # your Foursquare ID
CLIENT_SECRET = 'DK21E4U4POJOITFAIOOGX34YLDBGBMLDBGGID52EG0QBJUAN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZBO2RSEX4F1LTHDGY32VNYOOL1XAI1HFIF5125S0MIVAPAUQ
CLIENT_SECRET:DK21E4U4POJOITFAIOOGX34YLDBGBMLDBGGID52EG0QBJUAN


In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=900,LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
Northyork_venues = getNearbyVenues(names=NorthYork_data['Neighborhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [55]:
Northyork_venues.shape

(532, 7)

In [56]:
Northyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


In [58]:
Northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",28,28,28,28,28,28
Bayview Village,15,15,15,15,15,15
"Bedford Park, Lawrence Manor East",40,40,40,40,40,40
Don Mills,39,39,39,39,39,39
Downsview,64,64,64,64,64,64
"Fairview, Henry Farm, Oriole",38,38,38,38,38,38
Glencairn,25,25,25,25,25,25
Hillcrest Village,17,17,17,17,17,17
Humber Summit,6,6,6,6,6,6


In [63]:
print('There are {} distinct venues in {} categories.'.format(
    len(Northyork_venues['Venue'].unique()),len(Northyork_venues['Venue Category'].unique())))

There are 408 distinct venues in 142 categories.


### As, by above data table Willowdale, Willowdale East neighourhood has maximum number of venues in North York borough.

### Now analyse each neighbourhood

In [61]:
# one hot encoding
Northyork_onehot = pd.get_dummies(Northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Northyork_onehot['Neighborhood'] = Northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Northyork_onehot.columns[-1]] + list(Northyork_onehot.columns[:-1])
Northyork_onehot = Northyork_onehot[fixed_columns]

Northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bowling Alley,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Fireworks Store,Food & Drink Shop,Food Court,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,History Museum,Hockey Arena,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Nightclub,Office,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Ski Area,Ski Chalet,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [68]:
Northyork_grouped = Northyork_onehot.groupby('Neighborhood').mean().reset_index()
Northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bowling Alley,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Fireworks Store,Food & Drink Shop,Food Court,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,History Museum,Hockey Arena,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Nightclub,Office,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Ski Area,Ski Chalet,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000,0.035714,0.000000,0.000000,0.000000,0.00,0.035714,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.035714,0.000000,0.0000,0.035714,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.035714,0.035714,0.000000,0.00,0.035714,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,0.071429,0.00,0.00,0.00,0.000,0.000,0.00,0.000000,0.0000,0.000000,0.035714,0.000000,0.000000,0.035714,0.000000,0.000000,0.035714,0.000000,0.035714,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.035714,0.035714,0.000000,0.000000,0.000,0.000000,0.000000,0.035714,0.000000,0.00000,0.000000,0.000000,0.000,0.00
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.066667,0.000000,0.133333,0.000000,0.00,0.000000,0.000000,0.0000

In [69]:
num_top_venues = 10

for hood in Northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Northyork_grouped[Northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop  0.07
1          Pizza Place  0.07
2                 Bank  0.07
3       Sandwich Place  0.04
4           Ski Chalet  0.04
5             Ski Area  0.04
6          Gas Station  0.04
7        Shopping Mall  0.04
8  Fried Chicken Joint  0.04
9        Deli / Bodega  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.13
1          Gas Station  0.13
2                 Bank  0.13
3        Grocery Store  0.13
4                 Park  0.07
5         Intersection  0.07
6                 Café  0.07
7           Restaurant  0.07
8         Skating Rink  0.07
9        Shopping Mall  0.07


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.08
1       Italian Restaurant  0.08
2               Restaurant  0.05
3                     Park  0.05
4           Sandwich Place  0.05
5                     Bank  0.05
6    

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Northyork_grouped['Neighborhood']

for ind in np.arange(Northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pizza Place,Restaurant,Sandwich Place,Mediterranean Restaurant,Shopping Mall,Men's Store,Ski Area,Ski Chalet
1,Bayview Village,Bank,Grocery Store,Gas Station,Japanese Restaurant,Café,Restaurant,Chinese Restaurant,Skating Rink,Shopping Mall,Intersection
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Park,Restaurant,Bank,Sandwich Place,Juice Bar,Fast Food Restaurant,Liquor Store,Cosmetics Shop
3,Don Mills,Japanese Restaurant,Gym,Asian Restaurant,Office,Restaurant,Beer Store,Supermarket,Coffee Shop,History Museum,Middle Eastern Restaurant
4,Downsview,Grocery Store,Pizza Place,Vietnamese Restaurant,Coffee Shop,Hotel,Gas Station,Park,Sandwich Place,Pharmacy,Shopping Mall
5,"Fairview, Henry Farm, Oriole",Coffee Shop,Clothing Store,Bank,Restaurant,Japanese Restaurant,Chocolate Shop,Sandwich Place,Food Court,Shopping Mall,Burger Joint
6,Glencairn,Grocery Store,Fast Food Restaurant,Pizza Place,Coffee Shop,Gas Station,Sushi Restaurant,Playground,Pharmacy,Pet Store,Pub
7,Hillcrest Village,Pharmacy,Park,Bakery,Sandwich Place,Restaurant,Residential Building (Apartment / Condo),Korean Restaurant,Recreation Center,Chinese Restaurant,Bank
8,Humber Summit,Electronics Store,Park,Home Service,Arts & Crafts Store,Bakery,Discount Store,Fast Food Restaurant,Falafel Restaurant,Event Space,Dumpling Restaurant
9,"Humberlea, Emery",Convenience Store,Park,Gas Station,Golf Course,Discount Store,Bakery,Intersection,Dim Sum Restaurant,Diner,Dessert Shop


In [72]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              North Park, Maple Leaf Park, Upwood Park
1st Most Common Venue                            Convenience Store
2nd Most Common Venue                                  Coffee Shop
3rd Most Common Venue                                  Gas Station
4th Most Common Venue                                       Bakery
5th Most Common Venue                                         Park
6th Most Common Venue                             Department Store
7th Most Common Venue                                 Dessert Shop
8th Most Common Venue                           Dim Sum Restaurant
9th Most Common Venue                                        Diner
10th Most Common Venue                             Fireworks Store
Name: 11, dtype: object

### Cluster the North york Neighborhood using k-means

In [84]:

# set number of clusters
kclusters = 3

Northyork_grouped_clustering = Northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 2],
      dtype=int32)

In [85]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NorthYork_merged = NorthYork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
NorthYork_merged = NorthYork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NorthYork_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Bus Stop,Park,Pizza Place,Fast Food Restaurant,Food & Drink Shop,Café,Road,Laundry Service,Caribbean Restaurant,Supermarket
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Park,Portuguese Restaurant,Hockey Arena,Intersection,Sporting Goods Shop,Coffee Shop,Café,Discount Store,Event Space,Electronics Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Rental Car Location,Furniture / Home Store,Coffee Shop,Fast Food Restaurant,Dessert Shop,Vietnamese Restaurant,Restaurant,Arts & Crafts Store,Fried Chicken Joint
3,M3B,North York,Don Mills,43.745906,-79.352188,1,Japanese Restaurant,Gym,Asian Restaurant,Office,Restaurant,Beer Store,Supermarket,Coffee Shop,History Museum,Middle Eastern Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,1,Grocery Store,Fast Food Restaurant,Pizza Place,Coffee Shop,Gas Station,Sushi Restaurant,Playground,Pharmacy,Pet Store,Pub


In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NorthYork_merged['Latitude'], NorthYork_merged['Longitude'], NorthYork_merged['Neighborhood'], NorthYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### lets Examine Clusters

In [95]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 0, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Park,Portuguese Restaurant,Hockey Arena,Intersection,Sporting Goods Shop,Coffee Shop,Café,Discount Store,Event Space,Electronics Store
14,North York,0,Convenience Store,Coffee Shop,Gas Station,Bakery,Park,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Fireworks Store
19,North York,0,Convenience Store,Park,Gas Station,Golf Course,Discount Store,Bakery,Intersection,Dim Sum Restaurant,Diner,Dessert Shop
22,North York,0,Park,Convenience Store,Bowling Alley,Dog Run,French Restaurant,Restaurant,Tennis Court,Golf Course,Gym,Pet Store


In [96]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 1, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Bus Stop,Park,Pizza Place,Fast Food Restaurant,Food & Drink Shop,Café,Road,Laundry Service,Caribbean Restaurant,Supermarket
2,North York,1,Clothing Store,Rental Car Location,Furniture / Home Store,Coffee Shop,Fast Food Restaurant,Dessert Shop,Vietnamese Restaurant,Restaurant,Arts & Crafts Store,Fried Chicken Joint
3,North York,1,Japanese Restaurant,Gym,Asian Restaurant,Office,Restaurant,Beer Store,Supermarket,Coffee Shop,History Museum,Middle Eastern Restaurant
4,North York,1,Grocery Store,Fast Food Restaurant,Pizza Place,Coffee Shop,Gas Station,Sushi Restaurant,Playground,Pharmacy,Pet Store,Pub
5,North York,1,Japanese Restaurant,Gym,Asian Restaurant,Office,Restaurant,Beer Store,Supermarket,Coffee Shop,History Museum,Middle Eastern Restaurant
6,North York,1,Pharmacy,Park,Bakery,Sandwich Place,Restaurant,Residential Building (Apartment / Condo),Korean Restaurant,Recreation Center,Chinese Restaurant,Bank
7,North York,1,Bank,Coffee Shop,Pizza Place,Restaurant,Sandwich Place,Mediterranean Restaurant,Shopping Mall,Men's Store,Ski Area,Ski Chalet
8,North York,1,Coffee Shop,Clothing Store,Bank,Restaurant,Japanese Restaurant,Chocolate Shop,Sandwich Place,Food Court,Shopping Mall,Burger Joint
9,North York,1,Furniture / Home Store,Coffee Shop,Market,Fast Food Restaurant,Middle Eastern Restaurant,Massage Studio,Miscellaneous Shop,Caribbean Restaurant,Frame Store,Sports Bar
10,North York,1,Bank,Grocery Store,Gas Station,Japanese Restaurant,Café,Restaurant,Chinese Restaurant,Skating Rink,Shopping Mall,Intersection


In [97]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 2, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,2,Pool,Middle Eastern Restaurant,Ice Cream Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Electronics Store,Dumpling Restaurant,Dog Run,Discount Store
